In [ ]:
# -------------- Atmospheric River Detection Algorithm ---------------
# --------------------------------------------------------------------
# Author: Daemon Kennett (2021)
# 
# An Atmospheric River (AR) detection algorithm for global or
# regional studies. Note this code will be subject to 
# amendment and improvement over time.
# 
# The algorithm is a modified version of the Guan & Waliser (2015) method,
# implemented in python.
#
# If you use or adapt this python script please acknowledge the source, 
# and reference the GitHub repository at:
# https://github.com/daemonkennett/ar_detection
# 
# I am always interested in learning more about AR research.
# Please contact me at daemonkennett@gmail.com
# --------------------------------------------------------------------

In [ ]:
# ------------------- Import Modules and Packages --------------------
# --------------------------------------------------------------------
# Ignore deprecation warnings
import warnings
warnings.filterwarnings('ignore')
# Import modules and packages
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import csv
from datetime import date
from dateutil.relativedelta import relativedelta
import geopy.distance
import iris
import iris.quickplot as qplt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import os.path
from scipy import ndimage
from shapely.geometry import Point
import skimage
from skimage.segmentation import find_boundaries
import xarray as xr
# --------------------------------------------------------------------

In [ ]:
# ------------------------ Define Parameters -------------------------
# --------------------------------------------------------------------
# For the detection algorithm to run successfully, ensure that all 
# parameters in this cell are accurate.
# 
# Set file directories:
# Enter the directory containing this .ipynb script:
ar_dir = '/home/daemonkennett/Desktop/ar_detection_algorithm/' 
# Input data should be included in a subdirectory within the 
# ar_dir directory, named 'input_data'. The input_data directory 
# should include a separate IVT .nc file for each month, for both 
# eastward and northward IVT, using the format 'e-ivt-YYYY-#m' and 
# 'n-ivt-YYYY-#m' respectively, e.g. 'e-ivt-2021-1.nc'. A land mask
# .nc file should also be included in the ar_dir directory. This must
# be a .nc file of the same lat/lon shape as the IVT files, with a 
# single time dimension, and named land_mask.nc
input_data_dir = '{}input_data/'.format(ar_dir) # Names input data dir
#
# The netcdf files should be cropped to the desired lat/lon search 
# region before running the detection algorithm. This can be done for
# example by using ncks from the NCO package. 
# 
# Define search dates:
years = [2018,2019,2020] # List of years to include
months = [1,2,3,4,5,6,7,8,9,10,11,12] # List of months to include
# 
# Set custom parameters:
min_ivt = 150  # IVT threshold fixed lower limit (kg m**-1 s**-1)
threshold_percentage = 85  # Defines the IVT percentile threshold
min_length = 2000  # Minimum length of ARs in km
min_aspect = 2  # Minimum length/width ratio
min_span = 1000 # Minimum required distance between AR axis start/end points
# To include all ARs, not just landfalling ARs, set below value to 0,
# otherwise 1.
restrict_landfalling = 0
# To shorten the algorithm runtime, small objects that can not represent
# and AR are automatically discarded. The below value represents the minimum 
# size required of an AR, by number of grid cells. Enter a reasonable value
# depending on the resolution of the input data.
min_size = 60 # Minimum Size - Number of Grid Cells
# 
# Data to be output: (include '1' or exclude '0')
include_ar_snapshot = 1
include_ar_nc_masks = 1
include_ar_char_csv = 1
include_filtering_data = 1
#
# The following values must also be set, based on the size and resolution
# of the input data.
# 
# Enter the grid resolution of the data in degrees
res = 0.25
# Ener the input data lat/lon extent
max_lat = 90
min_lat = -90
max_lon = 180
min_lon = -180
# Enter the number of latitude/longitude grid cells of the input .nc files
lon_grid_number = 1440
lat_grid_number = 720
# Load Landfall Domain
land_mask_dir = '{}land_mask.nc'.format(ar_dir) # Names land mask file
land_mask = iris.load(land_mask_dir)[0][0]
if restrict_landfalling == 0:
    land_mask.data = ((land_mask.data * 0) + 1)
# Create a mask of the same shape as the input data, to indicate Northern and 
# Southern Hemispheres (convention 1 for NH and -1 for SH):
hemisphere = land_mask.copy() # Copy land mask
hemisphere.data[0:360]=1 # Set the range of NH grid values to 1
hemisphere.data[361:720]=-1 # Set the range of SH grid values to -1
# --------------------------------------------------------------------

In [ ]:
# ----------------------- Create Directories -------------------------
# --------------------------------------------------------------------
# This cell creates the subdirectories within the ar_dir directory.
if not os.path.exists('{}ivt_thresholds'.format(ar_dir)):
    os.makedirs('ivt_thresholds') # The IVT threshold .nc files are 
# saved within this directory to save repeatedly computing the IVT 
# thresholds on each seperate run of the algorithm. Note that these 
# .nc files must be deleted if changes are made to the IVT thresholds.
if not os.path.exists('{}ar_snapshots'.format(ar_dir)):
    os.makedirs('ar_snapshots') # Contains AR .png snapshots
if not os.path.exists('{}ar_masks'.format(ar_dir)):
    os.makedirs('ar_masks') # Contains AR shape .nc masks
if not os.path.exists('{}ar_axes'.format(ar_dir)):
    os.makedirs('ar_axes') # Contains AR axes .nc masks
if not os.path.exists('{}landfall_locations'.format(ar_dir)):
    os.makedirs('landfall_locations') # Contains max IVT cell .nc masks
if not os.path.exists('{}ar_characteristics'.format(ar_dir)):
    os.makedirs('ar_characteristics') # Contains AR characteristics csv file
with open('{}\
ar_characteristics/ar_characteristics.csv'.format(ar_dir), 'a', newline='') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['{}'.format('year'), \
                        '{}'.format('month'), \
                        '{}'.format('day'), \
                        '{}'.format('time'), \
                        '{}'.format('label'), \
                        '{}'.format('length'), \
                        '{}'.format('width'), \
                        '{}'.format('mean_ivt'), \
                        '{}'.format('mean_ivt_direction'), \
                        '{}'.format('landfall_ivt'), \
                        '{}'.format('landfall_ivt_direction')])
if not os.path.exists('{}filtering_data'.format(ar_dir)):
    os.makedirs('filtering_data') # Contains AR algorithm filtering csv
with open('{}\
filtering_data/filtering_data.csv'.format(ar_dir), 'a', newline='') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['{}'.format('year'), \
                        '{}'.format('month'), \
                        '{}'.format('num_ob_landfall'), \
                        '{}'.format('num_ob_size'), \
                        '{}'.format('num_ob_length'), \
                        '{}'.format('num_ob_narrowness'), \
                        '{}'.format('num_ob_poleward_ivt'), \
                        '{}'.format('num_ob_ivt_coherence'), \
                        '{}'.format('num_ob_orientation')])
# --------------------------------------------------------------------

In [ ]:
# --------------------- Run Detection Algorithm ----------------------
# --------------------------------------------------------------------
for year in years:
    for month in months:
        Date = date(year, month, 1)
        print('{}-{}'.format(year,month))
        # ---------------------------- Load Data -----------------------------
        # --------------------------------------------------------------------
        print('Loading Data')
        # Load 1 month of IVT data
        n_ivt_cubes = iris.load('{}n-ivt-{}-{}.nc'.format(input_data_dir,(Date).year, (Date).month))
        e_ivt_cubes = iris.load('{}e-ivt-{}-{}.nc'.format(input_data_dir,(Date).year, (Date).month))
        # Assign cubes to IVT variables
        northward_ivt = n_ivt_cubes[0]
        eastward_ivt = e_ivt_cubes[0]
        # Compute IVT magnitude (kg m**-1 s**-1)
        ivt = np.power(np.power(northward_ivt, 2) 
                       + np.power(eastward_ivt, 2), 1/2)
        zero = 0 * ivt[0] # Create zero cube with same shape as IVT data
        # Compute ivt direction within each cell
        ivt_direction = ivt.copy()
        ivt_direction.data = ((np.arctan2(eastward_ivt.data, northward_ivt.data) 
                               * 180 / np.pi) + 180) % 360
        # --------------------------------------------------------------------
        # ---------------------------- Sort Data -----------------------------
        ivt_list = []
        northward_ivt_list = []
        eastward_ivt_list = []
        ivt_direction_list = []
        for i in range(ivt.shape[0]):
            ivt_list.append(ivt[i])
            northward_ivt_list.append(northward_ivt[i])
            eastward_ivt_list.append(eastward_ivt[i])
            ivt_direction_list.append(ivt_direction[i])
        # ---------------------- Compute IVT Threshold -----------------------
        # Calculates percentile IVT over all time steps during 5 month 
        # period centred on target month (Month +/- 2 months).
        # The IVT threshold is defined as the maximum of percentile threshold
        # and the fixed lower limit.
        print('Computing IVT Thresholds')
        def compute_ivt_threshold(year, month):
            # Load 5 months of data (Target Month +/- 2 months)
            n_ivt_filename1 = '{}n-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=-2)).year, 
                                                      (Date+relativedelta(months=-2)).month)
            n_ivt_filename2 = '{}n-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=-1)).year, 
                                                      (Date+relativedelta(months=-1)).month)
            n_ivt_filename3 = '{}n-ivt-{}-{}.nc'.format(input_data_dir,(Date).year, (Date).month)
            n_ivt_filename4 = '{}n-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=+1)).year,
                                                      (Date+relativedelta(months=+1)).month)
            n_ivt_filename5 = '{}n-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=+2)).year,
                                                      (Date+relativedelta(months=+2)).month)
            n_ivt_filenames = [n_ivt_filename1, n_ivt_filename2, n_ivt_filename3,
                                n_ivt_filename4, n_ivt_filename5]
            n_ivt_xrcubes = xr.open_mfdataset(n_ivt_filenames)
            xr.Dataset.to_netcdf(n_ivt_xrcubes, 'n_ivt_data.nc')
            n_ivt_cubes0 = iris.load('n_ivt_data.nc')
            e_ivt_filename1 = '{}e-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=-2)).year, 
                                                      (Date+relativedelta(months=-2)).month)
            e_ivt_filename2 = '{}e-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=-1)).year, 
                                                      (Date+relativedelta(months=-1)).month)
            e_ivt_filename3 = '{}e-ivt-{}-{}.nc'.format(input_data_dir,(Date).year, (Date).month)
            e_ivt_filename4 = '{}e-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=+1)).year,
                                                      (Date+relativedelta(months=+1)).month)
            e_ivt_filename5 = '{}e-ivt-{}-{}.nc'.format(input_data_dir,
                                                        (Date+relativedelta(months=+2)).year,
                                                      (Date+relativedelta(months=+2)).month)
            e_ivt_filenames = [e_ivt_filename1, e_ivt_filename2, e_ivt_filename3,
                                e_ivt_filename4, e_ivt_filename5]
            e_ivt_xrcubes = xr.open_mfdataset(e_ivt_filenames)
            xr.Dataset.to_netcdf(e_ivt_xrcubes, 'e_ivt_data.nc')
            e_ivt_cubes0 = iris.load('e_ivt_data.nc')
            # Assign cubes to correct variable
            northward_ivt_0 = n_ivt_cubes0[0]
            eastward_ivt_0 = e_ivt_cubes0[0]
            # Compute IVT Magnitude (kg m**-1 s**-1)
            ivt_0 = 0 * northward_ivt_0
            ivt_0.data = np.power(np.power(northward_ivt_0.data, 2)
                           + np.power(eastward_ivt_0.data, 2), 1/2)
            zero_0 = 0 * ivt_0[0]
            # Compute IVT percentile threshold
            ivt_percentile_threshold = ivt_0.collapsed('time',
                                                   iris.analysis.PERCENTILE,
                                                   percent=threshold_percentage)
            # Define IVT lower limit (kg m**-1 s**-1)
            ivt_lower_limit = zero_0 + min_ivt
            # Compute IVT threshold
            ivt_threshold = (iris.analysis.maths.apply_ufunc(
                            np.maximum, ivt_percentile_threshold, ivt_lower_limit))
            return ivt_threshold
        # Save IVT threshold .nc files
        if os.path.exists('{}\
ivt_thresholds/ivt_threshold_{}-{}.nc'.format(ar_dir, year, month)) == True:
            ivt_threshold_cube_list = iris.load('{}\
ivt_thresholds/ivt_threshold_{}-{}.nc'.format(ar_dir, year, month))
            ivt_threshold = ivt_threshold_cube_list[0]
        else:
            ivt_threshold = compute_ivt_threshold(year, month)
            iris.save(ivt_threshold, '{}\
ivt_threshold_{}-{}.nc'.format(ar_dir, year, month))
        # --------------------------------------------------------------------
        # ------------------------ Identify Objects --------------------------
        print('Identifying Objects')
        # Identifies contiguous regions of enhanced IVT (Objects).
        # Create cube with value 1 if IVT is above threshold, 0 otherwise.
        object_mask = (iris.analysis.maths.apply_ufunc(np.greater, 
                                                       ivt, ivt_threshold))
        object_mask.data = object_mask.data.astype(int)
        # Create separate Cubes for each time step and compile as List.
        object_mask_list = []
        for i in range(ivt.shape[0]):
            object_mask_list.append(object_mask[i])
        # Label Objects and compile as a List.
        # The number of Objects identified for each time step is also recorded.
        labelled_object_list = []
        num_ob_list = []
        for i in range(ivt.shape[0]):
            label_cube = object_mask_list[i].copy()
            label_cube.data, num_ob = ndimage.label(label_cube.data)
            labelled_object_list.append(label_cube)
            num_ob_list.append(num_ob)
        # --------------------------------------------------------------------
        # ------------------------- Landfall Check ---------------------------
        # Objects that do not overlap the land mask are filtered.
        landfall_filter = []
        # Load Land Mask
        for i in range(ivt.shape[0]):
            Filter = []
            Array = land_mask.data * labelled_object_list[i].data
            for j in range(num_ob_list[i]):
                landfall_check = (j+1) in Array
                if landfall_check == False:
                    Filter.append(j+1)
            landfall_filter.append(Filter)
        # --------------------------------------------------------------------
        # --------------------------- Size Filter ----------------------------
        # Filter small Objects to speed up code run time.
        size_filter = []
        for i in range(ivt.shape[0]):
            Filter = []
            # Calculate sizes
            object_sizes = ndimage.sum(object_mask_list[i].data, 
                                       labelled_object_list[i].data, 
                                       list(range(1, num_ob_list[i]+1)))
            for j in range(num_ob_list[i]):
                if ((j+1) not in landfall_filter[i]):
                    if object_sizes[j] > min_size:
                        size_check = True
                    else:
                        size_check = False
                    if size_check == False:
                        Filter.append(j+1)
            size_filter.append(Filter)
        # --------------------------------------------------------------------
        # -------------------------- Compute Axis ----------------------------
        # --------------------------------------------------------------------
        print('Computing AR Axes')
        axis_list = []
        axis_coords_list = []
        landfall_locations = []
        landfall_ivt_magnitudes = []
        landfall_ivt_directions = []
        for i in range(ivt.shape[0]):
            # Create Cube of labelled Axis
            axis_cube = zero.copy()
            step_coords_list = []
            landfall_locations_cube = zero.copy()
            step_landfall_ivt_magnitudes = []
            step_landfall_ivt_directions = []
            # Compute Axis of each Object
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i])):
                    object_number = zero + (j+1)
                    object_mask = (iris.analysis.maths.apply_ufunc(
                                  np.equal, 
                                  labelled_object_list[i], 
                                  object_number))
                    object_ivt = object_mask * ivt_list[i]
                    landfall_ivt = object_ivt * land_mask
                    # Compute Axis
                    new_axis = zero.copy()
                    # Max IVT at landfall
                    max_ivt_coords = np.where((landfall_ivt.data.max() == 
                                               landfall_ivt.data) 
                                             & (landfall_ivt.data > 0))
                    max_ivt_coord = (np.array([max_ivt_coords[0][0]]), 
                                     np.array([max_ivt_coords[1][0]]))
                    axis_coords = [max_ivt_coord]
                    new_axis.data[max_ivt_coord[0], max_ivt_coord[1]] = 1
                    landfall_locations_cube.data = new_axis.data
                    step_landfall_ivt_magnitudes.append(ivt_list[i].data[max_ivt_coord[0], 
                                                                        max_ivt_coord[1]][0])
                    step_landfall_ivt_directions.append(ivt_direction_list[i].data[max_ivt_coord[0], 
                                                                                   max_ivt_coord[1]][0])
                    # Search Axis
                    while (object_ivt.data[max_ivt_coord[0], max_ivt_coord[1]] > 0).any():
                        if ((0 < max_ivt_coord[0][0] < lat_grid_number-1) and 
                            (0 < max_ivt_coord[1][0] < lon_grid_number-1)):
                            object_ivt.data[max_ivt_coord[0], max_ivt_coord[1]] = 0
                            # Compute adjacent cells
                            adjacent_cells = []
                            # N
                            north = (np.array([max_ivt_coord[0][0] - 1]), 
                                     np.array([max_ivt_coord[1][0]]))
                            # NE
                            north_east = (np.array([max_ivt_coord[0][0] - 1]), 
                                          np.array([max_ivt_coord[1][0] + 1]))
                            # E
                            east = (np.array([max_ivt_coord[0][0]]), 
                                    np.array([max_ivt_coord[1][0] + 1]))
                            # SE
                            south_east = (np.array([max_ivt_coord[0][0] + 1]), 
                                          np.array([max_ivt_coord[1][0] + 1]))
                            # S
                            south = (np.array([max_ivt_coord[0][0] + 1]), 
                                     np.array([max_ivt_coord[1][0]]))
                            # SW
                            south_west = (np.array([max_ivt_coord[0][0] + 1]), 
                                          np.array([max_ivt_coord[1][0] - 1]))
                            # W
                            west = (np.array([max_ivt_coord[0][0]]), 
                                    np.array([max_ivt_coord[1][0] - 1]))
                            # NW
                            north_west = (np.array([max_ivt_coord[0][0] - 1]), 
                                          np.array([max_ivt_coord[1][0] - 1]))
                            # Determine upstream direction
                            upstream_direction = ivt_direction[i].data[max_ivt_coord[0], 
                                                                       max_ivt_coord[1]]
                            if ((upstream_direction > 337.5) or (upstream_direction <= 22.5)):
                                list_of_new_coord = [north_west, north, north_east]
                            elif (22.5 < upstream_direction <= 67.5):
                                list_of_new_coord = [north, north_east, east]
                            elif (67.5 < upstream_direction <= 112.5):
                                list_of_new_coord = [north_east, east, south_east]
                            elif (112.5 < upstream_direction <= 157.5):
                                list_of_new_coord = [east, south_east, south]
                            elif (157.5 < upstream_direction <= 202.5):
                                list_of_new_coord = [south_east, south, south_west]
                            elif (202.5 < upstream_direction <= 247.5):
                                list_of_new_coord = [south, south_west, west]
                            elif (247.5 < upstream_direction <= 292.5):
                                list_of_new_coord = [south_west, west, north_west]
                            elif (292.5 < upstream_direction <= 337.5):
                                list_of_new_coord = [west, north_west, north]
                            ivt_values = [object_ivt.data[x[0], x[1]] 
                                          for x in list_of_new_coord]
                            index, value = max(enumerate(ivt_values), 
                                               key=operator.itemgetter(1))
                            max_ivt_coord = list_of_new_coord[index]
                            axis_coords.append(max_ivt_coord)
                            new_axis.data[max_ivt_coord[0], max_ivt_coord[1]] = 1
                        else:
                            object_ivt.data[max_ivt_coord[0], max_ivt_coord[1]] = 0
                    new_axis.data = new_axis.data.copy() * (j + 1)
                    axis_cube.data = axis_cube.copy().data + new_axis.data
                    step_coords_list.append(axis_coords)
                else:
                    step_coords_list.append([])
                    step_landfall_ivt_magnitudes.append(0)
                    step_landfall_ivt_directions.append(0)
            landfall_locations.append(landfall_locations_cube)
            axis_list.append(axis_cube)
            axis_coords_list.append(step_coords_list)
            landfall_ivt_magnitudes.append(step_landfall_ivt_magnitudes)
            landfall_ivt_directions.append(step_landfall_ivt_directions)
        # --------------------------------------------------------------------
        # ---------------------- Compute Axis Length -------------------------
        # --------------------------------------------------------------------
        axis_length_list = []
        def calc_length(axis_coords):
            length = 0
            for k in list(range(len(axis_coords)-1)):
                lat1=max_lat-(res*np.abs(axis_coords[k][0]))
                lon1=min_lon+(res*np.abs(axis_coords[k][1]))
                lat2=max_lat-(res*np.abs(axis_coords[k+1][0]))
                lon2=min_lon+(res*np.abs(axis_coords[k+1][1]))
                coords_1 = (lat1, lon1)
                coords_2 = (lat2, lon2)
                segment = geopy.distance.distance(coords_1, coords_2).km
                length += segment
            return length
        for i in range(ivt.shape[0]):
            # Compute length of each Object
            step_length_list = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i])):
                    length = calc_length(axis_coords_list[i][j])
                    step_length_list.append(length)
                else:
                    step_length_list.append(0)
            axis_length_list.append(step_length_list)
        # --------------------------------------------------------------------
        # ---------------------- Calculate Surface Areas ---------------------
        # --------------------------------------------------------------------
        # Surface area is in square kilometres.
        object_area_list = []
        # Compute surface area of each grid cell
        grid_areas = zero.copy()
        grid_areas.coord('latitude').guess_bounds()
        grid_areas.coord('longitude').guess_bounds()
        grid_areas.data = (iris.analysis.cartography.area_weights(grid_areas) 
                          / (1000**2))
        # Compute surface area of Objects
        for i in range(ivt.shape[0]):
            areas = ndimage.sum(grid_areas.data, 
                                   labelled_object_list[i].data, 
                                   list(range(1, num_ob_list[i]+1)))
            object_area_list.append(areas)
        # --------------------------------------------------------------------
        # -------------------------- Calculate Widths ------------------------
        # --------------------------------------------------------------------
        # The Width of an Object is calculated as its surface area divided
        # by its length.
        object_width_list = []
        for i in range(ivt.shape[0]):
            widths = []
            for j in range(num_ob_list[i]):
                if (axis_length_list[i][j] > 0):
                    width = object_area_list[i][j] / axis_length_list[i][j]
                    widths.append(width)
                else:
                    width = 0
                    widths.append(width)
            object_width_list.append(widths)
        # --------------------------------------------------------------------
        # -------------------------- AR Criteria -----------------------------
        # --------------------------------------------------------------------
        # -------------------- Criterion 1: Length Check ---------------------
        print('Length Criterion')
        # Filter Objects based on axis length.
        filter_list_1 = []
        for i in range(ivt.shape[0]):
            Filter = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i])):
                    length_check = (axis_length_list[i][j] > min_length)
                    if length_check == False:
                        Filter.append(j+1)
            filter_list_1.append(Filter)
        # --------------------------------------------------------------------

        # ------------------ Criterion 2: Narrowness Check -------------------
        # Filter Objects based on length/width ratio.
        print('Narrowness Criterion')
        filter_list_2 = []
        for i in range(ivt.shape[0]):
            Filter = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i]) 
                    & ((j+1) not in filter_list_1[i])):
                    narrowness_check = ((axis_length_list[i][j] 
                                        / object_width_list[i][j]) > min_aspect)
                    if narrowness_check == False:
                        Filter.append(j+1)
            filter_list_2.append(Filter)
        # --------------------------------------------------------------------
        # ----------------- Create List of Max IVT Coords --------------------
        max_IVT_coords_list = []
        for i in range(ivt.shape[0]):
            NewList = []
            for j in range(num_ob_list[i]):
                if (len(axis_coords_list[i][j]) > 0):
                    coords_A = axis_coords_list[i][j][0]
                else:
                    coords_A = (0,0)
                NewList.append(coords_A)
            max_IVT_coords_list.append(NewList)
        # --------------------------------------------------------------------
        # ------------ Create List of Object Mean IVT Magnitude --------------
        mean_ivt_magnitude_list = []
        for i in range(ivt.shape[0]):
            NewList = ndimage.mean(ivt_list[i].data, 
                                   labelled_object_list[i].data, 
                                   list(range(1, num_ob_list[i]+1)))
            mean_ivt_magnitude_list.append(NewList)
        # --------------------------------------------------------------------
        # ------------ Create List of Object Mean IVT Direction --------------
        mean_ivt_direction_list = []
        for i in range(ivt.shape[0]):
            mean_northward_ivt = ndimage.mean(northward_ivt[i].data, 
                                              labelled_object_list[i].data, 
                                              list(range(1, num_ob_list[i]+1)))
            mean_eastward_ivt = ndimage.mean(eastward_ivt[i].data, 
                                             labelled_object_list[i].data, 
                                             list(range(1, num_ob_list[i]+1)))
            NewList = ((np.arctan2(mean_eastward_ivt, mean_northward_ivt) 
                        * 180 / np.pi) + 180) % 360
            mean_ivt_direction_list.append(NewList)
        # --------------------------------------------------------------------
        # --------------- Create List of Object Orientations -----------------
        object_orientation_list = []
        for i in range(ivt.shape[0]):
            NewList = []
            for j in range(num_ob_list[i]):
                if (len(axis_coords_list[i][j]) > 0):
                    coords_A = axis_coords_list[i][j][0]
                    coords_B = axis_coords_list[i][j][-1]
                else:
                    coords_A = (0,0)
                    coords_B = (0,0)
                orientation = ((np.arctan2(coords_A[1] - coords_B[1], 
                                           coords_A[0] - coords_B[0]) 
                            * 180 / np.pi) + 180) % 360
                NewList.append(orientation)
            object_orientation_list.append(NewList)
        # --------------------------------------------------------------------
        # -------- Create List of Distances between Axis Start and End -------
        axis_distance_list = []
        for i in range(ivt.shape[0]):
            NewList = []
            for j in range(num_ob_list[i]):
                if (len(axis_coords_list[i][j]) > 0):
                    coords_A = axis_coords_list[i][j][0]
                    coords_B = axis_coords_list[i][j][-1]
                else:
                    coords_A = (0,0)
                    coords_B = (0,0)
                lat1=max_lat-(res*np.abs(coords_A[0]))
                lon1=min_lon+(res*np.abs(coords_A[1]))
                lat2=max_lat-(res*np.abs(coords_B[0]))
                lon2=min_lon+(res*np.abs(coords_B[1]))
                coords_1 = (lat1, lon1)
                coords_2 = (lat2, lon2)
                axis_distance = geopy.distance.distance(coords_1, coords_2).km
                NewList.append(axis_distance)
            axis_distance_list.append(NewList)
        # --------------------------------------------------------------------
        #-------------- Criterion 3: Mean Meridional IVT Check ---------------
        # An object is discarded if the mean IVT does not have a poleward 
        # component > 50 kg m**-1 s**-1.
        print('Meridional IVT Criterion')
        filter_list_3 = []
        for i in range(ivt.shape[0]):
            Filter = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i]) 
                    & ((j+1) not in filter_list_1[i]) 
                    & ((j+1) not in filter_list_2[i])):
                    mean_poleward_ivt = (ndimage.mean(northward_ivt_list[i].data * 
                                                      hemisphere.data, 
                                                   labelled_object_list[i].data, 
                                                    j+1))
                    mean_meridional_ivt_check = mean_poleward_ivt > 50
                    if mean_meridional_ivt_check == False:
                        Filter.append(j+1)
            filter_list_3.append(Filter)
        # --------------------------------------------------------------------
        # ---------- Criterion 4: Coherence in IVT Direction Check -----------
        # If more than half of the grid cells have IVT deviating more than 45°
        # from the object’s mean IVT, the object is filtered.
        print('Coherence IVT Direction Criterion')
        filter_list_4 = []
        for i in range(ivt.shape[0]):
            Filter = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i]) 
                    & ((j+1) not in filter_list_1[i])
                    & ((j+1) not in filter_list_2[i])
                    & ((j+1) not in filter_list_3[i])):
                    mean_direction = mean_ivt_direction_list[i][j]
                    deviation_from_mean_direction = (iris.analysis.maths.apply_ufunc(
                        np.absolute, ivt_direction_list[i] - mean_direction))
                    percentage_coherence_ivt_direction = (ndimage.mean(
                        np.logical_or(deviation_from_mean_direction.data < 45, 
                                      deviation_from_mean_direction.data > 315), 
                        labelled_object_list[i].data, 
                        j+1))
                    coherence_ivt_check = percentage_coherence_ivt_direction > 0.5
                    if coherence_ivt_check == False:
                        Filter.append(j+1)
            filter_list_4.append(Filter)
        # --------------------------------------------------------------------
        # Criterion 5: Consistency between IVT Direction and Object Orientation
        # If object orientation deviates from the mean IVT direction by more than 
        # 45 degrees, the object is filtered. Object orientation is calculated as
        # the angle between the first and last grid cells of the AR axis. Further,
        # the distance between the first and last grid cells of the AR axis must be
        # greater than 1000 km.
        print('Consistent Orientation Criterion')
        filter_list_5 = []
        for i in range(ivt.shape[0]):
            Filter = []
            for j in range(num_ob_list[i]):
                if (((j+1) not in size_filter[i]) 
                    & ((j+1) not in landfall_filter[i]) 
                    & ((j+1) not in filter_list_1[i])
                    & ((j+1) not in filter_list_2[i])
                    & ((j+1) not in filter_list_3[i])
                    & ((j+1) not in filter_list_4[i])):
                    mean_direction = mean_ivt_direction_list[i][j]
                    object_orientation = object_orientation_list[i][j]
                    deviation_from_mean_direction = np.absolute(float(mean_direction)- 
                                                                float(object_orientation))
                    consistent_orientation_check = deviation_from_mean_direction > 45
                    axis_distance_check = axis_distance_list[i][j] < min_span
                    if (consistent_orientation_check or axis_distance_check) == False:
                        Filter.append(j+1)
            filter_list_5.append(Filter)
        # --------------------------------------------------------------------
        #------------------------ Create AR Snapshot -------------------------
        # --------------------------------------------------------------------
        print('Saving Data')
        if include_ar_snapshot == 1:
            for i in range(ivt.shape[0]):
                if num_ob_list[i] > 0:
                    Count = 0
                    for j in range(num_ob_list[i]):
                        if (((j+1) not in size_filter[i]) 
                        & ((j+1) not in landfall_filter[i]) 
                        & ((j+1) not in filter_list_1[i])
                        & ((j+1) not in filter_list_2[i])
                        & ((j+1) not in filter_list_3[i])
                        & ((j+1) not in filter_list_4[i])
                        & ((j+1) not in filter_list_5[i])):
                            Count += 1
                            date_text = str(ivt[i].coord('time'))[10:20]
                            time_text = str(ivt[i].coord('time'))[21:23]
                            lat1=max_lat-(res*np.abs(int(max_IVT_coords_list[i][j][0])))
                            lon1=min_lon+(res*np.abs(int(max_IVT_coords_list[i][j][1])))
                            AR_coords = (lat1, lon1)
                            # Add a matplotlib axes, specifying the display projection
                            ax = plt.axes(projection 
                                          = ccrs.PlateCarree(central_longitude=180+AR_coords[1]))
                            ax.set_extent((-50, 50, AR_coords[0]-30, AR_coords[0]+30), 
                                          crs = ccrs.PlateCarree(central_longitude=180+AR_coords[1]))
                            ax.outline_patch.set_visible(False)
                            # Add coastlines
                            ax.coastlines(resolution='50m', 
                                          color='black', 
                                          linewidth=0.5)
                            # Plot AR Boundary
                            colors2 = ["#FFFFFF00", "#20ff00"]
                            cmap2 = matplotlib.colors.ListedColormap(colors2)
                            object_number = zero + (j+1)
                            object_mask = (iris.analysis.maths.apply_ufunc(
                                          np.equal, 
                                          labelled_object_list[i], 
                                          object_number))
                            boundary_obj = object_mask.copy()
                            iris.plot.contour(boundary_obj, levels=1, linewidths=0.5,
                                              cmap=cmap2, zorder=20)
                            # Plot IVT contours
                            ivt_contour = (iris.plot.contourf(
                                ivt[i], levels=np.linspace(250,1500,11), 
                                cmap=matplotlib.cm.get_cmap('Blues'), 
                                zorder=0, extend='max'))
                            # Plot AR axis
                            colors1 = ["#FFFFFF00", "#FFFF00"]
                            cmap1= matplotlib.colors.ListedColormap(colors1)
                            axis_obj = axis_list[i].copy()
                            axis_obj.data = np.where(axis_obj.data == j + 1, 1, 0)
                            iris.plot.contourf(axis_obj, 1, cmap = cmap1, zorder=20)
                            # Plot IVT vectors
                            uwind = eastward_ivt[i]
                            vwind = northward_ivt[i]
                            ulon = uwind.coord('longitude')
                            vlon = vwind.coord('longitude')
                            ulon.points = ulon.points - 180.0
                            vlon.points = vlon.points - 180.0
                            x = ulon.points
                            y = vwind.coord('latitude').points
                            u = uwind.data
                            v = vwind.data
                            plt.quiver(x[::12], y[::12], u[::12,::12], v[::12,::12], 
                                       pivot='mid', scale = 45000, color='gray', 
                                       zorder=5, width = 0.001)
                            # Plot colorbar
                            cbar = plt.colorbar(ivt_contour, shrink = 0.55)
                            cbar.set_label('IVT (kg/m/s)', rotation=270, labelpad = 10, fontsize=8)
                            # Plot title
                            plt.title("Atmospheric River Detected at {}-{} UTC".format(date_text,
                                                                                       time_text), 
                                      fontdict = {'fontsize' : 10})
                            # Plot AR statistics
                            length = str(round(axis_length_list[i][j], -1))[:-1][:-1]
                            width = str(round(object_width_list[i][j], -1))[:-1][:-1]
                            mean_ivt_magnitude = (str(round(mean_ivt_magnitude_list[i][j], 
                                                          0))[:-1][:-1])
                            mean_ivt_direction = (str(round(mean_ivt_direction_list[i][j], 
                                                          1))[:-1][:-1])
                            landfall_ivt_magnitude = (str(round(landfall_ivt_magnitudes[i][j], 
                                                          0))[:-1][:-1])
                            landfall_ivt_direction = (str(round(landfall_ivt_directions[i][j], 
                                                          1))[:-1][:-1])
                            textstr = "Date: {} UTC \nLength: {} km, Width: {} km" \
                            "\nMean IVT Magnitude: {} kg/m/s, Mean IVT Direction: {}°" \
                            "\nLandfall IVT Magnitude: {} kg/m/s, Landfall IVT Direction: {}°" \
                            "\nObject Boundary: Green, Axis: Yellow".format(date_text, 
                                                                            length, 
                                                                            width, 
                                                                            mean_ivt_magnitude, 
                                                                            mean_ivt_direction, 
                                                                            landfall_ivt_magnitude, 
                                                                            landfall_ivt_direction)
                            # These are matplotlib.patch.Patch properties
                            props = dict(boxstyle='Square', facecolor='skyblue', alpha=0.3)
                            # Place a text box
                            ax.text(0, -0.15, textstr, transform=ax.transAxes, fontsize=6,
                                    verticalalignment='top', bbox=props)
                            # Save Figure
                            plt.savefig('{}\
ar_snapshots/ar_{}-{}-UTC_snapshot_({})'.format(ar_dir, date_text, time_text, Count), 
                                        dpi=700)
                            plt.close("all")
            plt.close("all")
        # --------------------------------------------------------------------
        # --------------------------- Save Data ------------------------------
        # --------------------------------------------------------------------
        # ------------------------- Save AR Masks ----------------------------
        if include_ar_nc_masks == 1:
            for i in range(ivt.shape[0]):
                date_text = str(ivt[i].coord('time'))[10:20]
                time_text = str(ivt[i].coord('time'))[21:23]
                if num_ob_list[i] > 0:
                    Count = 0
                    for j in range(num_ob_list[i]):
                        if (((j+1) not in size_filter[i]) 
                        & ((j+1) not in landfall_filter[i]) 
                        & ((j+1) not in filter_list_1[i])
                        & ((j+1) not in filter_list_2[i])
                        & ((j+1) not in filter_list_3[i])
                        & ((j+1) not in filter_list_4[i])
                        & ((j+1) not in filter_list_5[i])):
                            Count += 1
                            ar_mask = labelled_object_list[i].copy()
                            ar_axes = zero.copy()
                            landfall_location = zero.copy()
                            ar_mask.data = np.where(ar_mask.data == j + 1, 1, 0)
                            iris.save(ar_mask, '{}\
ar_masks/ar_{}-{}-UTC_mask_({}).nc'.format(ar_dir, date_text, time_text, Count))
                            iris.save(axis_list[i], '{}\
ar_axes/ar_{}-{}-UTC_axes_({}).nc'.format(ar_dir, date_text, time_text, Count))
                            iris.save(landfall_locations[i], '{}\
landfall_locations/ar_{}-{}-UTC_landfall_location_({}).nc'.format(ar_dir, date_text, time_text, Count))
        # --------------------------------------------------------------------
        # ------------------------ Save AR Statistics ------------------------
        # Save AR characteristics as .csv file
        if include_ar_char_csv == 1:
            for i in range(ivt.shape[0]):
                if num_ob_list[i] > 0:
                    Count = 0
                    for j in range(num_ob_list[i]):
                        if (((j+1) not in size_filter[i]) 
                        & ((j+1) not in landfall_filter[i]) 
                        & ((j+1) not in filter_list_1[i])
                        & ((j+1) not in filter_list_2[i])
                        & ((j+1) not in filter_list_3[i])
                        & ((j+1) not in filter_list_4[i])
                        & ((j+1) not in filter_list_5[i])):
                            Count += 1
                            date_text = str(ivt[i].coord('time'))[10:20]
                            time_text = str(ivt[i].coord('time'))[21:23]
                            year = date_text[0:4]
                            month = date_text[5:7]
                            day = date_text[8:10]
                            time = time_text
                            length = str(round(axis_length_list[i][j], -1))[:-1][:-1]
                            width = str(round(object_width_list[i][j], -1))[:-1][:-1]
                            mean_ivt = (str(round(mean_ivt_magnitude_list[i][j], 
                                                          0))[:-1][:-1])
                            mean_ivt_direction = (str(round(mean_ivt_direction_list[i][j], 
                                                          1))[:-1][:-1])
                            landfall_ivt = (str(round(landfall_ivt_magnitudes[i][j], 
                                                          0))[:-1][:-1])
                            landfall_ivt_direction = (str(round(landfall_ivt_directions[i][j], 
                                                          1))[:-1][:-1])
                            with open('{}\
/ar_characteristics/ar_characteristics.csv'.format(ar_dir), 'a', newline='') as f:
                                thewriter = csv.writer(f)
                                thewriter.writerow(['{}'.format(year), \
                                                    '{}'.format(month), \
                                                    '{}'.format(day), \
                                                    '{}'.format(time), \
                                                    '{}'.format(Count), \
                                                    '{}'.format(length), \
                                                    '{}'.format(width), \
                                                    '{}'.format(mean_ivt), \
                                                    '{}'.format(mean_ivt_direction), \
                                                    '{}'.format(landfall_ivt), \
                                                    '{}'.format(landfall_ivt_direction)])
        # -------------------- Save Filtering Statistics ---------------------
        # --------------------------------------------------------------------
        if include_filtering_data == 1:
            num_landfall_filter = []
            num_size_filter = []
            num_filter_1 = []
            num_filter_2 = []
            num_filter_3 = []
            num_filter_4 = []
            num_filter_5 = []
            [num_landfall_filter.append(len(x)) for x in landfall_filter]
            [num_size_filter.append(len(x)) for x in size_filter]
            [num_filter_1.append(len(x)) for x in filter_list_1]
            [num_filter_2.append(len(x)) for x in filter_list_2]
            [num_filter_3.append(len(x)) for x in filter_list_3]
            [num_filter_4.append(len(x)) for x in filter_list_4]
            [num_filter_5.append(len(x)) for x in filter_list_5]
            num_ob_0 = np.sum(num_ob_list)
            num_ob_1 = num_ob_0 - np.sum(num_landfall_filter) - np.sum(num_size_filter)
            num_ob_2 = num_ob_1 - np.sum(num_filter_1) 
            num_ob_3 = num_ob_2 - np.sum(num_filter_2) 
            num_ob_4 = num_ob_3 - np.sum(num_filter_3)
            num_ob_5 = num_ob_4 - np.sum(num_filter_4)
            num_ob_6 = num_ob_5 - np.sum(num_filter_5)
            with open('{}\
filtering_data/filtering_data.csv'.format(ar_dir), 'a', newline='') as f:
                thewriter = csv.writer(f)
                thewriter.writerow(['{}'.format(year), \
                                    '{}'.format(month), \
                                    '{}'.format(num_ob_0), \
                                    '{}'.format(num_ob_1), \
                                    '{}'.format(num_ob_2), \
                                    '{}'.format(num_ob_3), \
                                    '{}'.format(num_ob_4), \
                                    '{}'.format(num_ob_5), \
                                    '{}'.format(num_ob_6)])
        # --------------------------------------------------------------------
        # --------------------------------------------------------------------